In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T

In [2]:
resnet = models.resnet50(pretrained=True)

In [19]:
for params in resnet.parameters():
    params.requires_grad_(False)
modules = list(resnet.children())[:-2]
resnet = nn.Sequential(*modules)

In [20]:
x = torch.randn(4, 3, 224, 224)

In [21]:
features = resnet(x)

In [22]:
features.shape

torch.Size([4, 2048, 7, 7])

In [24]:
features = features.permute(0, 2, 3, 1)
features.shape

torch.Size([4, 7, 7, 2048])

In [25]:
features = features.view(features.shape[0], -1, features.shape[-1])
features.shape

torch.Size([4, 49, 2048])